Before we start, let's make sure we got access to a GPU:

Edit->Notebook Setting->Hardware Accelerator

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# ML Coding example #2: Atomization Energies with PyTorch (Feed-Forward Neural Network)

The QM7 dataset contains XYZ structures for 7101 small molecules, with up to 7 atoms of type CNO, saturated with H.

Some of them look like this:
![alt text](https://i.imgur.com/vQymf39.png)

Attempt to map all organic molecules with up to 7 CNO-atoms.

For each molecule you will be given the atomization energy calculated using a QM method (PBE0/def2-TZVP).


*   Instead of the raw 7101 XYZ files, we will use the Bag-of-Bonds features for each molecule, along with the atomization energies.

References: 

1.   Rupp et al. (2012) Phys Rev Lett https://doi.org/10.1103/PhysRevLett.108.058301
2.   Hansen et al. (2015) J Phys Chem Lett https://doi.org/10.1021/acs.jpclett.5b00831



In [0]:
!wget -O bob.npy https://www.dropbox.com/s/vyiwza2uy4jkczg/bob.npy
!wget -O hof.npy https://www.dropbox.com/s/zy717f8mwxaegff/hof.npy

Load the data into numpy arrays and plot the distribution of energies:

In [0]:
import numpy as np

Xall = np.load("bob.npy")
Yall = np.load("hof.npy")


import matplotlib.pyplot as plt
plt.hist(Yall)
plt.xlabel("Atomization Energy [kcal/mol]")
plt.ylabel("Counts")
plt.show()

The values are very large and negative, which can be a problem for neural networks. To avoid this, we can normalize the data or scale all the target values by a factor.

Scale the values by the mean of the dataset and divide into Training and Test sets:

In [0]:
n_train = 4000
n_valid = 1000
n_test  = 2000

Yall = np.load("hof.npy")
y_mean = np.mean(Yall)
print(y_mean)
Yall /= y_mean

Xtrain = Xall[:n_train]
Xvalid = Xall[n_train:n_valid+n_train]
Xtest  = Xall[-n_test:]

Ytrain = Yall[:n_train]
Yvalid = Yall[n_train:n_valid+n_train]
Ytest  = Yall[-n_test:]

Plotting again!

In [0]:
plt.hist(Yall, label="All")
plt.hist(Ytrain, label="Training")
plt.hist(Ytest, label="Test")
plt.hist(Yvalid, label="Validation")

plt.xlabel("Target value")
plt.ylabel("Counts")
plt.legend()
plt.show()

Wondering what our representations look like? Print a row in `Xtrain`!

In [0]:
print(Xtrain[1])
print(len(Xtrain[1]))

## Defining out Architecture:
Four layers (including input and output), so this qualifies for the buzzword "Deep Learning":
<img src="https://i.imgur.com/dsEGsVg.png" width="400">

Fully-connected neural network with two hidden layer in PyTorch:


In [0]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden1_size, hidden2_size):

    super(NeuralNet, self).__init__()

    # Define the operations in the layers
    # Here "Linear", i.e: y = a*X.T + b
    self.fc1 = nn.Linear(input_size,hidden1_size)
    self.fc2 = nn.Linear(hidden1_size,hidden2_size)
    self.fc3 = nn.Linear(hidden2_size, 1)

  def forward(self, x):

    # Define the activation functions,
    # i.e. the "forward" passes
    x = torch.sigmoid(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    x = self.fc3(x)

    return x


Sigmoid functions again:\begin{equation}
\sigma\left(x\right) = \frac{1}{1 + \exp\left(-x\right)}
\end{equation}

Linear connections between the layers:
\begin{equation}
y = b + \sum_i x_i  w_i
\end{equation}




Since we are using a GPU, all the training and test data must be moved to the GPU:

In [0]:
dtype = torch.float
device = torch.device("cuda:0")

x = torch.from_numpy(Xtrain).to(device, dtype)
y = torch.from_numpy(Ytrain.reshape((n_train,1))).to(device, dtype)

xv = torch.from_numpy(Xvalid).to(device, dtype)
yv = torch.from_numpy(Yvalid.reshape((n_valid,1))).to(device, dtype)

xs = torch.from_numpy(Xtest).to(device, dtype)
ys = torch.from_numpy(Ytest.reshape((n_test,1))).to(device, dtype)

## Optimizing the Neural Network weights:

First, lets define the neural network, and the sizes of the hidden layers. Secondly, we have to send it to the GPU.

In [0]:
input_size = 465

# 30 & 5 are good values
# 64 & 50 also worth a try
hidden1_size = 30
hidden2_size = 5

# Define our model
model = NeuralNet(input_size, hidden1_size, hidden2_size)

# Send it to the GPU
model = model.to(device)

Check parameters:

In [0]:
for param in model.parameters():
  print(param.shape)
  print(param)

Next, we have to select a loss function. For $y = sin(x)$ we minimized the squared error:

\begin{equation}
L_2\left(\mathbf{w}\right) = \sum_{i=1}^{N} \left(y_i^\mathrm{true} -  y_i^\mathrm{predicted} \right)^{2}
\end{equation}
From experience, the L1-loss function (mean absolute error) works better for this experiment:
\begin{equation}
L_1\left(\mathbf{w}\right) = \frac{1}{N}\sum_{i=1}^{N} | y_i^\mathrm{true} -  y_i^\mathrm{predicted} |
\end{equation}

Lastly, we also need to define the optimizer. In this case we choose the "Adam" optimizer, which is a variant of gradient-descent optimization.
*  Kingma & Ba (2014) *arXiv*, https://arxiv.org/abs/1412.6980

In [0]:
# L2 loss function (Mean Squared Error)
# loss_fn = nn.MSELoss() 

# L1 loss (Mean Absoulte Error)
loss_fn = nn.L1Loss()

# Optimizer, lr = learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Now, we are write the optimization procedure

1.   Make prediction
2.   Calculate the loss
3.   Calculate the gradient of the loss function
4.   Take a step in the direction suggested by the gradient.

For example 30000 epoch (about 30-60 sec on Google Colab):

In [0]:
for epoch in range(30000):

  # Make a prediction with the model
  y_predicted = model.forward(x)
 
  # Calculate the Loss-function
  loss = loss_fn(y_predicted, y)

  # Set gradient to zero
  optimizer.zero_grad()

  # Take the gradient of the loss function
  loss.backward()

  # Take a step with the optimizer
  optimizer.step()

  
  # Print additional aoutput
  if (epoch % 1000 == 0):
    mse =(y - y_predicted).pow(2).sum()
    rmsd = torch.sqrt(mse/n_train) * abs(y_mean)
    mae = (y - y_predicted).abs().sum() * abs(y_mean) / n_train
    
    yvs = model.forward(xv)
    rmsd_v = torch.sqrt((yvs - yv).pow(2).sum()/n_valid) * abs(y_mean)
    mae_v = (yvs - yv).abs().sum() * abs(y_mean) / n_valid
    
    mae = mae.to("cpu").detach().numpy()
    rmsd = rmsd.to("cpu").detach().numpy()
    mae_v = mae_v.to("cpu").detach().numpy()
    rmsd_v = rmsd_v.to("cpu").detach().numpy()
    
    print("EPOCH %7i    MAE(train) = %7.2f   RMSE(train) = %7.2f   MAE(valid) = %7.2f   RMSE(valid) = %7.2f    [kcal/mol]" %
        (epoch, mae, rmsd, mae_v, rmsd_v))


### Calcluate test errors:
With the optimized neural network, we can now predict the error on the test set. 

Because everything is stored on the GPU, we also have to copy back to the CPU and convert to Numpy array for convenience:

In [0]:
# Calculate test, copy back to CPU and convert to numpy 
y_pred = model(xs).to("cpu").detach().numpy().flatten() * y_mean

# Copy true test values back to CPU and convert to numpy
y_true = ys.to("cpu").detach().numpy().flatten() * y_mean

print(y_pred)
print(y_true)

mae = np.mean(np.abs(y_pred - y_true))
rmse = np.sqrt(np.mean(np.square(y_pred - y_true)))

print("MAE:", mae)
print("RMSE:", rmse)

Lastly, lets plot a correlation plot between the true and predicted atomization energies:

In [0]:

import matplotlib.pyplot as plt


plt.scatter(y_true,y_pred)
plt.xlabel("True Atomization Energy [kcal/mol]")
plt.ylabel("Predicted Atomization Energy [kcal/mol]")
plt.show()

### Note on hyper parameters and optimization

In order to have the best performing neural network, we could have changed a lot of things:

*   Number of hidden layers
*   Hidden layer sizes
*   Type of loss function
*   Optimizer + setting
*   Regularization of parameters


If you are wondering, how we could have added regularization to the loss function, here is an example for example L2 regularization:

In [0]:
l2reg = 1e-4

for param in model.parameters():
    loss += torch.norm(param) * l2reg